In [203]:
import warnings
from typing import List, Optional, Dict
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_groq import ChatGroq
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

from langgraph.graph import END, StateGraph
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

warnings.filterwarnings("ignore", category=FutureWarning)

`1. Initializing Elastic search client, LLM model connection and Embedding model`

In [204]:
es_client = Elasticsearch(
    hosts="https://512baa73334c4127ade77bc2dfa2ef02.eastus2.azure.elastic-cloud.com:443",
    basic_auth=("elastic", "XRzYOABuG17Yv32UKAFySw41")
)

print(f"Elastic Search Connection successful: {es_client.ping()}")

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    # api_key="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",
    api_key="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"
)

print("LLM Model loaded")

model_name = "nomic-ai/nomic-embed-text-v1"
model = SentenceTransformer(model_name, trust_remote_code=True)

print("Embedding Model loaded")

def embedding_creation(input_texts, model):
    embedding = model.encode(input_texts)
    return embedding.tolist()

Elastic Search Connection successful: True
LLM Model loaded


<All keys matched successfully>


Embedding Model loaded


In [205]:
print(len(embedding_creation("Hello", model)))

768


`2. RAG`

`2.1 Multi Query Generation`

In [385]:
query = "round water related planning in Abu Dhabi"

In [403]:
class MultiQuerySchema(BaseModel):
    Questions: List[str] = Field(description="List of questions")


MULTI_QUERY_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI, specialized in generating effective multi-queries for Retrieval Augmented Generation (RAG) systems.

    Guidelines:
    1. Analyze the user's original query and generate a set of five distinct yet relevant search queries that cover:
        - Synonyms and similar terms
        - Related subtopics or specific aspects
        - Broader and narrower variations of the original query
    2. Aim to create queries that will increase the chances of retrieving diverse, relevant results by considering different interpretations and possible user intents.
    3. Provide JSON response, with list of questions like {{ "Questions": ["Question 1", "Question 2", ..] }}
    4. Provide only the list of queries without any additional explanations or commentary.

    User Query: "{user_query}"
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_query"],
)

structured_llm = llm.with_structured_output(MultiQuerySchema, method="json_mode")

multi_query_generation = MULTI_QUERY_PROMPT | structured_llm

In [404]:
multi_query_response = multi_query_generation.invoke({"user_query": query})
multi_query_response.Questions

['Water management strategies in Abu Dhabi',
 'Roundabouts in Abu Dhabi waterways',
 'Water conservation planning in Abu Dhabi',
 'Circular economy initiatives for water in Abu Dhabi',
 'Urban planning for water resources in Abu Dhabi']

['Groundwater management strategies in Abu Dhabi', 'Water resource planning in Abu Dhabi', 'Abu Dhabi groundwater conservation initiatives', 'Water supply planning for Abu Dhabi', 'Groundwater sustainability in the Emirate of Abu Dhabi']

`2.2 Agent: Enriching Queries with Synonyms`

In [405]:
# 1. Query Enriching Agent

QUERY_ENRICHING_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI assistant specialized in generating Elasticsearch query strings. 
    Your task is to create the most effective query string for the given user question. 
    This query string will be used to search for relevant documents in an Elasticsearch index.

    Guidelines:
    1. Analyze the user's question carefully.
    2. Generate ONLY a query string suitable for Elasticsearch's match query.
    3. Focus on key terms and concepts from the question.
    4. Include synonyms or related terms that might be in relevant documents.
    5. Use simple Elasticsearch query string syntax if helpful (e.g., OR, AND).
    6. Do not use advanced Elasticsearch features or syntax.
    7. Do not include any explanations, comments, or additional text.
    8. Provide only the query string, nothing else.

    For the question "What is Clickthrough Data?", we would expect a response like:
    clickthrough data OR click-through data OR click through rate OR CTR OR user clicks OR ad clicks OR search engine results OR web analytics

    AND operator is not allowed. Use only OR.

    User Question: {user_query}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_query"],
)

query_enricher = QUERY_ENRICHING_PROMPT | llm | StrOutputParser()

In [418]:
QUERY_ENRICHING_PROMPT.save("test.yaml")

In [406]:
query_text = query_enricher.invoke({"user_query": multi_query_response.Questions})
enriched_query = [term.strip() for term in query_text.split(' OR ')]
enriched_query

['water management',
 'water conservation',
 'waterways',
 'roundabouts',
 'water resources',
 'circular economy',
 'urban planning',
 'water conservation planning',
 'Abu Dhabi']

`Embedding creation`

In [407]:
quesry_embedding = embedding_creation(query, model)

In [408]:
class DatasourceResponseSchema(BaseModel):
    """Datascore name"""
    datasource: str = Field(description="Name of the datascore")

In [409]:
KNOWLEDGEBASE_IDENTIFICATION_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI Retrieval Augmented Generation Agent, specialized in identifying the best data source for answering the given user question.

    Guidelines:
    1. Analyze the given question and identify one amoung the following data source based on its description
        - Elasticsearch: This knowledge base contains comprehensive information about Abu Dhabi's reviewing rules, policies, plans, priorities, and legislation.
    2. If the query is vague, focus on the most prominent keywords to guide your choice.
    3. Ensure that your selection is the most relevant based on the definitions provided.
    4. Provide only the data score name without any additional explanations or commentary.
    
    User Query: "{user_query}"
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["user_query"],
)

In [410]:
structured_llm = llm.with_structured_output(DatasourceResponseSchema)
data_score_chain = KNOWLEDGEBASE_IDENTIFICATION_PROMPT | structured_llm 
data_score_chain.invoke({"user_query": user_query})

DatasourceResponseSchema(datasource='Elasticsearch')

In [411]:
# from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_core.tools import tool
# from typing import List, Dict, Any

In [412]:
# from typing import List
# from pydantic import BaseModel, Field

# class ElasticsearchInputSchema(BaseModel):
#     index_name: str = Field(..., description="The name of the index to search")
#     query_keys: List[str] = Field(..., description="The keys to search in the text fields.")
#     query_vector: List[float] = Field(..., description="The vector representation of the query.")
#     text_fields: List[str] = Field(..., description="The text fields to match against.")
#     vector_field: str = Field(..., description="The name of the field containing vector data.")
#     num_candidates: int = Field(100, description="Number of candidates to consider (default: 100).")
#     num_results: int = Field(10, description="Number of results to return (default: 10).")


# @tool(args_schema=ElasticsearchInputSchema)
# def elasticsearch(
#     index_name: str,
#     query_keys: str,
#     query_vector: List[float] = None,
#     text_fields: List[str] = None,
#     vector_field: str = "embedded_content",
#     num_candidates: int = 100,
#     num_results: int = 10,
#     ) -> str:
#     """
#     Performs elasticsearch in Elasticsearch knowledgebase

#     Args:
#         index_name (str): The name of the index to search.
#         query_keys (List[str]): The keys to search in the text fields.
#         query_vector (List[float]): The vector representation of the query.
#         text_fields (List[str]): The text fields to match against.
#         vector_field (str): The name of the field containing vector data.
#         num_candidates (int): Number of candidates to consider (default: 100).
#         num_results (int): Number of results to return (default: 10).

#     Returns:
#         str: Retrieved docuemnts from the elastic search.
#     """

#     if not isinstance(query_keys, list) or not all(isinstance(k, str) for k in query_keys):
#         raise ValueError("query_keys must be a list of strings.")
#     if not isinstance(query_vector, list) or not all(isinstance(v, float) for v in query_vector):
#         raise ValueError("query_vector must be a list of floats.")
#     if not isinstance(text_fields, list) or not all(isinstance(f, str) for f in text_fields):
#         raise ValueError("text_fields must be a list of strings.")
    
#     if text_fields is None:
#         text_fields = ["content", "heading", "questions", "reference"]

#     search_body = {
#         "knn": {
#             "field": vector_field,
#             "query_vector": query_vector,
#             "k": num_candidates,
#             "num_candidates": num_candidates
#         },
#         "query": {
#             "bool": {
#                 "must": [
#                     {
#                         "multi_match": {
#                             "query": " ".join(query_keys),
#                             "fields": text_fields,
#                             "type": "best_fields",
#                             "operator": "or"
#                         }
#                     }
#                 ],
#                 "should": [
#                     {
#                         "script_score": {
#                             "query": {"match_all": {}},
#                             "script": {
#                                 "source": """
#                                 double vector_score = cosineSimilarity(params.query_vector, params.vector_field) + 1.0;
#                                 double text_score = _score;
#                                 return 0.7 * vector_score + 0.3 * text_score;
#                                 """,
#                                 "params": {
#                                     "query_vector": query_vector,
#                                     "vector_field": vector_field
#                                 }
#                             }
#                         }
#                     }
#                 ]
#             }
#         }
#     }

#     conn = Elasticsearch(
#         hosts="https://512baa73334c4127ade77bc2dfa2ef02.eastus2.azure.elastic-cloud.com:443",
#         basic_auth=("elastic", "XRzYOABuG17Yv32UKAFySw41")
#     )

#     response = conn.search(index=index_name, body=search_body, size=num_results)

#     print(response)

#     result = []
#     all_content = ""
#     for j in response['hits']['hits']:
#         content = j["_source"]
#         _result = {}
#         _result["score"] = j["_score"]
#         _result["heading"] = content['heading']
#         _result["content"] = content['content']
#         all_content = all_content + '\n' + content['content']
#         _result["reference"] = content['reference']

#         result.append(_result)

#     return str(result)

# tools=[elasticsearch]

# agent = create_tool_calling_agent(llm, tools, KNOWLEDGEBASE_IDENTIFICATION_PROMPT)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True,)

# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "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"
# os.environ["LANGCHAIN_PROJECT"] = "New"

# response = agent_executor.invoke(
#     {
#         "tools": tools,
#         "query_keys": enriched_query,
#         "query_vector": quesry_embedding,
#         "user_query": query
#     }
# )

`2.3 Tool: Hybrid Search`

In [413]:
def hybrid_vector_search(
        conn,
        index_name: str, 
        query_keys: str, 
        query_vector: List[float], 
        text_fields: List[str], 
        vector_field: str, 
        num_candidates: int = 100, 
        num_results: int = 10,
    ) -> Dict:


    search_body = {
            "knn": {
                "field": vector_field,
                "query_vector": query_vector,
                "k": num_candidates,
                "num_candidates": num_candidates
            },
            "query": {
                "bool": {
                    "must": [
                        {
                            "multi_match": {
                                "query": " ".join(query_keys),  
                                "fields": text_fields,
                                "type": "best_fields",
                                "operator": "or" 
                            }
                        }
                    ],
                    "should": [
                        {
                            "script_score": {
                                "query": {"match_all": {}},
                                "script": {
                                    "source": """
                                    double vector_score = cosineSimilarity(params.query_vector, params.vector_field) + 1.0;
                                    double text_score = _score;
                                    return 0.7 * vector_score + 0.3 * text_score;
                                    """,
                                    "params": {
                                        "query_vector": query_vector,  
                                        "vector_field": vector_field
                                    }
                                }
                            }
                        }
                    ]
                }
            }
        }

    response = conn.search(index=index_name, body=search_body, size=num_results)

    return response, search_body


In [417]:
print(enriched_query)

['water management', 'water conservation', 'waterways', 'roundabouts', 'water resources', 'circular economy', 'urban planning', 'water conservation planning', 'Abu Dhabi']


In [414]:
# # test_query_vector = embedding_creation(query)

['groundwater management', 'groundwater conservation', 'water resource planning', 'water supply planning', 'groundwater sustainability', 'water management', 'Abu Dhabi', 'Abu Dhabi Emirate', 'Abu Dhabi Emirate water', 'Abu Dhabi groundwater', 'Abu Dhabi water supply', 'Abu Dhabi water resources', 'Abu Dhabi sustainability']
['water management', 'water conservation', 'waterways', 'roundabouts', 'water resources', 'circular economy', 'urban planning', 'water conservation planning', 'Abu Dhabi']


In [445]:
query_vector = embedding_creation(query, model)

response, search_body = hybrid_vector_search(
        es_client,
        index_name="abudhabi-policies-legislations-v4", 
        query_keys= enriched_query,
        query_vector=query_vector, 
        text_fields=["content", "heading", "questions", "reference"], 
        vector_field="embedded_content", 
        num_candidates = 100, 
        num_results = 3
    )

result = []
all_content = ""
for j in response['hits']['hits']:
    content = j["_source"]
    _result = {}
    _result["score"] = j["_score"]
    _result["heading"] = content['heading']
    _result["content"] = content['content']
    all_content = all_content + '\n' + content['content']
    _result["reference"] = content['reference']

    result.append(_result)

result

C:\Users\tshar\AppData\Local\Temp\ipykernel_11376\2427214058.py:54: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = conn.search(index=index_name, body=search_body, size=num_results)


[{'score': 38.973217,
  'heading': 'executive council resolution no. (182) of 2022 concerning the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi',
  'content': "executive council resolution no. (182) of 2022 concerning the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi the executive council has decided the following: the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi shall be approved in accordance with the (attached form). issuance date: 13 december 2022 corresponding to: 19 jumada al owla 1444 hijrisaif saeed ghobash secretary-general \x03 classification: confidential \x03 ©sader\x03 \x03 \x03 \x03 \x03 \x03  \x03 general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi environment agency - abu dhabi. 2022. 1. introduction this policy reviews the existing state of groun

In [447]:

sorted_data_desc = sorted(result, key=lambda x: x['score'])
sorted_data_desc

[{'score': 19.237661,
  'heading': 'law no. (6) of 2023 concerning the reorganisation of abu dhabi future energy company "public joint stock company"',
  'content': 'law no. (6) of 2023 concerning the reorganisation of abu dhabi future energy company "public joint stock company" we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1998 concerning the regulation of the water and electricity sector in the emirate of abu dhabi, and its amendments; • law no. (22) of 2007 concerning the establishment of abu dhabi future energy company (private joint stock company) and the establishment of masdar institute of science and technology, and its amendments; • law no. (1) of 2017 concerning the financial system of abu dhabi government, and its amendments; • law no. (2) of 2017 concerning the establishment of mubadala investment company, ‘pub

`2.4 Agent: Extracting Reference Document details`

In [422]:
def reference_search(conn, index_name, reference_index):
    search_body = {
        "size": 1,
        "query": {
            "bool": {
                "must": [
                    {
                    "match": {"heading": reference_index}
                    }
                ]
            }
        }
    }
    response = conn.search(index=index_name, body=search_body)

    return response
    

ref_doc = list()
for doc in result:
    for reference_index in doc["reference"]:
        response = reference_search(es_client, "abudhabi-policies-legislations-v4", reference_index)
        result = []
        for j in response['hits']['hits']:
            content = j["_source"]
            _result = {}
            _result["score"] = j["_score"]
            _result["heading"] = content['heading']
            _result["content"] = content['content']
            _result["reference"] = content['reference']

            ref_doc.append(_result)

In [433]:
ref_doc

[{'score': 12.823528,
  'heading': 'law no. (5) of 2023 concerning the regulation of the trade of petroleum products in the emirate of abu dhabi',
  'content': 'law no. (5) of 2023 concerning the regulation of the trade of petroleum products in the emirate of abu dhabi we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi, and its amendments; • law no. (11) of 2018 concerning the establishment of the department of energy; • law no. (24) of 2020 concerning the supreme council for financial and economic affairs; • federal law no. (14) of 2017 concerning the trade of petroleum products; • chairman of executive council resolution no. (26) of 2018 concerning additional competencies for the department of energy; and • based on what was proposed to an

`2.5 Agent: Critic Agent`

In [431]:
CRITIC_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are the AI agent specialized in Review the given document and assess its relevance to the user"s query or not.
  
    Guidelines:
    1. Analyze the user's question and identify key terms and concepts against the document.
    2. Consider synonyms and related terms that may appear in relevant documents.
    3. Provide binary response like yes or no
    4. Do not include explanations or additional text.

    Docuemnt title: {title}
    Docuemnt content: {content}
    User Question: {question}

    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "title", "content"],  
)

relevency_checking = CRITIC_PROMPT | llm

In [434]:
CRITIC_PROMPT.save("doc_relevancy.yaml")

In [432]:
for doc in ref_doc:
    response = relevency_checking.invoke(
    {
        "title": doc['heading'],
        "question": query, 
        "content": doc['content'],

    }
    )
    print(response.content)

No
Yes
No
No
No
No
No
Yes
No
Yes
No
Yes
No
No
No
No
No
No
No
No
No
No
No
No


In [158]:
reference_doc = list()

for doc in ref_doc:
    reference_doc.append(doc['heading'])

reference_doc

['executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities',
 'executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities',
 'law no. (1) of 2022 concerning the amendment of some provisions of law no. (1) of 2017 concerning the financial system of abu dhabi government',
 'law no. (1) of 2022 concerning the amendment of some provisions of law no. (1) of 2017 concerning the financial system of abu dhabi government',
 'chairman of the executive council resolution no. (4) of 2021 concerning the reformation of the board of directors of abu dhabi housing authority',
 'executive council resolution no. (131) of 2021 concerning the reformation of the board of trustees of the emirates college for advanced education',
 'executive council resolution no. (135) of 2021 concerning the membership of the board of trustees of khalif

In [159]:
response = relevency_checking.invoke(
    {
        "question": query, 
        "documents": reference_doc
    }
)
print(response.content)

{
  "executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities": true,
  "executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities": true,
  "law no. (1) of 2022 concerning the amendment of some provisions of law no. (1) of 2017 concerning the financial system of abu dhabi government": false,
  "law no. (1) of 2022 concerning the amendment of some provisions of law no. (1) of 2017 concerning the financial system of abu dhabi government": false,
  "chairman of the executive council resolution no. (4) of 2021 concerning the reformation of the board of directors of abu dhabi housing authority": false,
  "executive council resolution no. (131) of 2021 concerning the reformation of the board of trustees of the emirates college for advanced education": false,
  "executive council resolution no. (135) of 2021 concerning 

In [160]:
import json
conditional_response = json.loads(response.content)
conditional_response

{'executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities': True,
 'law no. (1) of 2022 concerning the amendment of some provisions of law no. (1) of 2017 concerning the financial system of abu dhabi government': False,
 'chairman of the executive council resolution no. (4) of 2021 concerning the reformation of the board of directors of abu dhabi housing authority': False,
 'executive council resolution no. (131) of 2021 concerning the reformation of the board of trustees of the emirates college for advanced education': False,
 'executive council resolution no. (135) of 2021 concerning the membership of the board of trustees of khalifa university of science and technology': False,
 'executive council resolution no. (136) of 2021 concerning the reformation of the board of trustees of sorbonne university abu dhabi': False,
 'executive council resolution no. (137) of 2021 concerning the appointment of

In [161]:
for topic, value in conditional_response.items():
    if value:
        all_content += f'\n{topic}: {value}'

`2.6 Agent: Response creation`

In [162]:
# Define the prompt template
prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI legal assistant chatbot designed to answer user questions based on provided legal documents.

    Response Instructions:
    1. Provide a concise, informative response directly addressing the users question.
    2. Cite specific sections or clauses from the documents to support your answer.
    3. If the documents lack sufficient information, respond with "I don't have knowledge to answer your question."
    4. Avoid starting responses with phrases like "Based on the provided documents..."
    5. Do not include explanations or additional context.
    6. Include any relevant quantitative measures.

    User Question: {question}
    Legal Documents: {documents}

    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "documents"],
)

# # Create the question router
# question_router = prompt | llm | StrOutputParser() 

# response = question_router.invoke(
#     {
#         "question": query, 
#         "documents": all_content  # Use the corrected variable name
#     }
# )

In [438]:
import json

with open('response.json', 'r') as file:
    data = json.load(file)


In [444]:
data["response"]

'[{\'score\': 95.178246, \'heading\': \'executive council resolution no. (182) of 2022 concerning the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi\', \'content\': "executive council resolution no. (182) of 2022 concerning the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi the executive council has decided the following: the general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi shall be approved in accordance with the (attached form). issuance date: 13 december 2022 corresponding to: 19 jumada al owla 1444 hijrisaif saeed ghobash secretary-general \\x03 classification: confidential \\x03 ©sader\\x03 \\x03 \\x03 \\x03 \\x03 \\x03  \\x03 general policy for the management, regulation and protection of groundwater in the emirate of abu dhabi environment agency - abu dhabi. 2022. 1. introduction this policy reviews the existing 

In [163]:
print(response)

The members of the Board of Trustees of Mohamed bin Zayed University for Humanities are:

1. H.E. Dr. Mohammed Rashid Ahmed Al Hamli - Chairman
2. Mubarak Hamad Mubarak Al Muhairi - Vice Chairman
3. Mohammed Hamza Hassan Al Qassim
4. Maryam Eid Khamis Al Muhairi
5. Abdullah Aqeela Ali Al Muhairi
6. Dr. Omar Habtoor Theeb Al-Derei
7. Ghanem Sultan Ahmed Al Suwaidi

The membership term of the Board shall be three renewable years. (Executive Council Resolution No. (11) of 2023, Clause 2)


In [167]:
response = question_router.invoke(
    {
        "question": query, 
        "documents": all_content  # Use the corrected variable name
    }
)

for s in question_router.stream(    {
        "question": query, 
        "documents": all_content  # Use the corrected variable name
    }
):
    print(s, end="", flush=True)

The members of the Board of Trustees of Mohamed bin Zayed University for Humanities are:

1. H.E. Dr. Mohammed Rashid Ahmed Al Hamli - Chairman
2. Mubarak Hamad Mubarak Al Muhairi - Vice Chairman
3. Mohammed Hamza Hassan Al Qassim
4. Maryam Eid Khamis Al Muhairi
5. Abdullah Aqeela Ali Al Muhairi
6. Dr. Omar Habtoor Theeb Al-Derei
7. Ghanem Sultan Ahmed Al Suwaidi

The membership term of the Board shall be three renewable years. (Executive Council Resolution No. (11) of 2023, Clause 2)

In [166]:
s

''

In [448]:
content = "The policy for the management, regulation, and protection of groundwater in Abu Dhabi aims to ensure the sustainability of groundwater reserves for future generations. \n\nThe policy is based on several main principles, including the encouragement of consumers to use alternative water sources, conservation and efficient use of groundwater, allocation of agricultural lands in non-restricted areas with long-term sustainable water resources, and education of consumers about the economic value of groundwater.\n\nThe policy objectives include ensuring the optimal use of groundwater, achieving a comprehensive knowledge of groundwater resources, and promoting the use of good irrigation techniques that reduce groundwater consumption.\n\nThe policy is implemented through a set of regulatory and non-regulatory tools, including the Environment Agency - Abu Dhabi, in coordination with the Department of Energy and Abu Dhabi Authority for Agriculture and Food Safety, and in consultation with the authorities concerned with groundwater in the emirate.\n\nThe expected results of applying the policy include a decrease of groundwater extraction by 650 million cubic meters by 2030, improvement of the groundwater quality index, decrease of the rate of water extraction versus the rate of recharge from 24 to 16 times by 2030, and the use of recycled water to enhance the recharge of the aquifer in the areas used for irrigation.\n\nThe estimated economic value of each cubic meter of groundwater in Abu Dhabi ranges between AED 7.7 and AED 9, and the estimated total groundwater in Abu Dhabi ranges between AED 15.5 and AED 18 billion annually.\n\nThe policy aims to address the challenges of groundwater depletion, which has led to a decrease in groundwater levels and a deterioration of its quality in many areas, with 79% of it becoming highly saline, 18% medium brackish water, and only 3% considered fresh water.\n\nThe policy also aims to address the economic and social effects of groundwater depletion, including the need to provide alternatives of fresh water as a strategic reserve, the reliance on small desalination plants in farms, and the negative effects on agricultural products and the income and returns of farmers and farm owners.\n\nThe policy is expected to achieve positive environmental impacts, including the decrease of groundwater extraction, improvement of the groundwater quality index, and the use of recycled water to enhance the recharge of the aquifer in the areas used for irrigation."

print(content)

The policy for the management, regulation, and protection of groundwater in Abu Dhabi aims to ensure the sustainability of groundwater reserves for future generations. 

The policy is based on several main principles, including the encouragement of consumers to use alternative water sources, conservation and efficient use of groundwater, allocation of agricultural lands in non-restricted areas with long-term sustainable water resources, and education of consumers about the economic value of groundwater.

The policy objectives include ensuring the optimal use of groundwater, achieving a comprehensive knowledge of groundwater resources, and promoting the use of good irrigation techniques that reduce groundwater consumption.

The policy is implemented through a set of regulatory and non-regulatory tools, including the Environment Agency - Abu Dhabi, in coordination with the Department of Energy and Abu Dhabi Authority for Agriculture and Food Safety, and in consultation with the authoriti